In [ ]:
# default_exp experiment

# Experiment utils

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export
from pathlib import Path

import numpy as np
import pandas as pd
from fastai.basics import *

from vqa.utils import generate_time_id
from vqa.wrangling import prepare_train_dataframe

In [ ]:
#export
def log_training_dataset(df, wandb_run=None):
    train_dataframe = df[['video_name', 'frames', 'scene', 'label', 'distortion', 'severity', 'is_valid']]
    path = 'train_dataframe.json'
    train_dataframe.to_json(path, orient='records')
    if wandb_run:
        import wandb
        artifact = wandb.Artifact('train_dataframe', type='dataset')
        artifact.add_file(path)
        wandb_run.log_artifact(artifact)
        wandb.log(dict(
            df=wandb.Table(dataframe=train_dataframe),
        ))
    return path

In [ ]:
#export
import json

def log_model_evaluation(clf_report, scores, wandb_run=None):
    clf_report_path = 'classification_report.txt'
    with open(clf_report_path, 'w') as f:
        f.write(clf_report)
    
    scores_path = 'scores.json'
    with open(scores_path, 'w') as f:
        json.dump(scores, f)
    
    if wandb_run:
        import wandb

        artifact = wandb.Artifact('classification_report', type='perf')
        artifact.add_file(clf_report_path)
        wandb_run.log_artifact(artifact)

        artifact = wandb.Artifact('scores', type='perf')
        artifact.add_file(scores_path)
        wandb_run.log_artifact(artifact)

        wandb.config.update(scores)
    
    return clf_report_path, scores_path

In [ ]:
#export
def log_preds_for_competition(preds, wandb_run=None):
    path = 'predict.txt'
    with open(path, 'w') as f:
        f.write('\n'.join(preds))
    if wandb_run:
        import wandb
        artifact = wandb.Artifact('test-predictions', type='perf')
        artifact.add_file(path)
        wandb_run.log_artifact(artifact)
    return path

In [ ]:
#export
def make_experiment_dir(root='./experiments', name=None):
    name = name or generate_time_id()
    experiment_dir = Path(root) / name
    experiment_dir.mkdir(parents=True, exist_ok=True)
    return experiment_dir

In [ ]:
#export
def make_dataframes(train_dataframe_path, train_dir, tst_dir, frame_indices_list, drop_reference):
    df = pd.read_json(train_dataframe_path)
    df['label'] = df['label'].replace('R_0', 'R0_0') # todo: remove this line after updating dataset on kaggle
    df = prepare_train_dataframe(df, train_dir, frame_indices_list, drop_reference)
    tst_df = make_test_dataframe(tst_dir, frame_indices_list)
    return df, tst_df